We re-create numerical artefacts in the implementation of GPR with a periodic kernel for forward prediction. 
We consider a simple truth trajectory given by a deterministic sine curve. Next, we configure the GPRP algorithm and plot predictions. 

To get started, I import some python software I will need:

In [1]:
import sys
sys.path.append('../../../')
from plot_tools.fig_preamble import * # imports numpy, matplotlib, Py2PDF, plot_helper_funcs, plot_figstyle_sheet
import GPy


from gpr.common import get_data, simple_unlearnable_sine
from data_tools.load_raw_cluster_data import LoadExperiment as le
from plot_tools.plot_figstyle_sheet import color_pallete, predmarker, COLOURDICT, STYLEDICT
from analysis_tools.common import sqr_err

/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1206: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markerfacecolor != fc:
/home/riddhisw/anaconda2/lib/python2.7/site-packages/matplotlib/lines.py:1182: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if self._markeredgecolor != ec:


I also set up filepaths for saving data

In [2]:
ver=0
datapath = '../../DATA_v'+str(ver)+'_/' #v'+str(ver)+'/DATA_v'+str(ver)+'/'
datafile = 'GPR_BR_calc_RBF_RQ_MAT32'
test_case=19
variation=5

In [3]:
dataobject = le(test_case, variation, 
                skip = 1,
                GPRP_load='No', GPRP_path = './',
                LKFFB_load = 'Yes', LKFFB_path = datapath,
                AKF_load='No', AKF_path = './',
                LSF_load = 'No', LSF_path = './')

n_train = dataobject.Expt.n_train
Delta_T = dataobject.Expt.Delta_T_Sampling
n_predict = dataobject.Expt.n_predict
n_testbefore = dataobject.Expt.n_testbefore

LKFFB: Data Loaded? Yes


In [4]:
print dataobject.Expt.n_train, dataobject.Expt.Delta_T_Sampling, dataobject.Expt.n_predict, dataobject.Expt.n_testbefore

2000 0.001 100 50


In [5]:
# Set initial starting values for L-BFGS-B OPtimiser in GPy
R_0 =  100 # cannot be unity since RQ fails to initialise # 1.0
total_runs=50
kernel_name = ['RBF', 'RQ', 'MAT32']
total_opt_params = 1 # total number of Gpy Optimisation params

We now implement optimised GPRP model but with constrained periodicities

In [6]:
pred_list= np.zeros((len(kernel_name), total_runs, n_predict+n_testbefore))
opt_list = np.zeros((len(kernel_name), total_runs, total_opt_params))
truth_list= np.zeros((len(kernel_name), total_runs, n_predict+n_testbefore))
failed_runs_list = np.zeros((len(kernel_name), total_runs))

In [7]:
for idx_run in xrange(total_runs): 

    X, Y, TestX, truth, msmts = get_data(dataobject, points=n_train, randomize='y')
  
    for idx_ker in xrange(len(kernel_name)):        
        
        # Reset
        KERNEL=0
        gauss=0
        exact=0
        m1=0

        # Built model
        KERNEL = {'RBF': GPy.kern.RBF(1), 'RQ': GPy.kern.RatQuad(1), 'MAT32': GPy.kern.sde_Matern32(1)}
        gauss = GPy.likelihoods.Gaussian(variance=R_0)
        exact = GPy.inference.latent_function_inference.ExactGaussianInference()
        
        
        print("Optimise GPR for:", kernel_name[idx_ker], idx_run)
        print(m1)
        
        try:
            m1 = GPy.core.GP(X=X, Y=Y, kernel=KERNEL[kernel_name[idx_ker]], likelihood=gauss, 
                             inference_method=exact)
            m1.optimize(optimizer=None, messages=False)
            # opt_list[idx_per, idx_run, :] = [m1.Gaussian_noise.variance[0]]
            pred_list[idx_ker, idx_run, :] = m1.predict(TestX)[0].flatten()
            print("Optimisation complete!")
            print(m1)

        except:
            # opt_list[idx_per, idx_run, :] = np.zeros(total_opt_params)
            pred_list[idx_ker, idx_run, :] = np.zeros(int(TestX.shape[0]))
            failed_runs_list[idx_ker, idx_run] = idx_run
            print("Unexpected error:", sys.exc_info()[0])
            print("Failed run", idx_run)

        truth_list[idx_ker, idx_run, :] = truth[int(TestX[0]):] # duplications, but downstream code is easier
        
        np.savez(datapath+datafile,
                 truth_list=truth_list, 
                 pred_list=pred_list,
                 # opt_list=opt_list, 
                 failed_runs_list=failed_runs_list,
                 kernel_name=kernel_name,
                 n_train=n_train, n_predict=n_predict, n_testbefore=n_testbefore,
                 R_0=R_0, 
                 Delta_T=Delta_T, total_runs=total_runs, test_case=test_case, variation=variation )


('Optimise GPR for:', 'RBF', 0)
0


 ../../../gpr/common.py:66: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future
 ../../../gpr/common.py:67: VisibleDeprecationWarning:using a non-integer number instead of an integer will result in an error in the future


Optimisation complete!

Name : gp
Objective : 4767.15801174
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |               value  |  constraints  |  priors
  rbf.variance             |       717929.113958  |      +ve      |        
  rbf.lengthscale          |    0.00461262818743  |      +ve      |        
  Gaussian_noise.variance  |  5.56268464627e-309  |      +ve      |        
('Optimise GPR for:', 'RQ', 0)
0


 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:160: RuntimeWarning:overflow encountered in divide
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:570: RuntimeWarning:overflow encountered in square
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:575: RuntimeWarning:overflow encountered in square
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:577: RuntimeWarning:invalid value encountered in multiply
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:582: RuntimeWarning:overflow encountered in square
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:584: RuntimeWarning:invalid value encountered in multiply
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/paramz/transformations.py:106: RuntimeWarning:invalid value encountered in greater
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/par

('Unexpected error:', <class 'numpy.linalg.linalg.LinAlgError'>)
('Failed run', 0)
('Optimise GPR for:', 'MAT32', 0)
0
Optimisation complete!

Name : gp
Objective : 3991.51058903
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |               value  |  constraints  |  priors
  Mat32.variance           |       590104.078718  |      +ve      |        
  Mat32.lengthscale        |       2.41365771587  |      +ve      |        
  Gaussian_noise.variance  |  5.56268464627e-309  |      +ve      |        
('Optimise GPR for:', 'RBF', 1)
0
Optimisation complete!

Name : gp
Objective : 14193.4738344
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |          value  |  constraints  |  priors
  rbf.variance             |  767279.870676  |      +ve      |        
  rbf.lengthscale          |  7.59685250133  |      +ve      |        
  Gaussian_noise.variance  |  

...

Optimisation complete!

Name : gp
Objective : 18885.3674072
Number of Parameters : 4
Number of Optimization Parameters : 4
Updates : True
Parameters:
  gp.                      |          value  |  constraints  |  priors
  RatQuad.variance         |  14858.0498843  |      +ve      |        
  RatQuad.lengthscale      |   200.83401239  |      +ve      |        
  RatQuad.power            |   951.00390157  |      +ve      |        
  Gaussian_noise.variance  |  14938.8025193  |      +ve      |        
('Optimise GPR for:', 'MAT32', 3)
0
Optimisation complete!

Name : gp
Objective : 4710.66612827
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |               value  |  constraints  |  priors
  Mat32.variance           |       749203.085513  |      +ve      |        
  Mat32.lengthscale        |    0.00352190320373  |      +ve      |        
  Gaussian_noise.variance  |  5.56268464627e-309  |      +ve      |        
('Opt

 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:403: RuntimeWarning:overflow encountered in multiply
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/stationary.py:403: RuntimeWarning:invalid value encountered in multiply


Optimisation complete!

Name : gp
Objective : 17755.3644802
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |          value  |  constraints  |  priors
  Mat32.variance           |  22564.2294815  |      +ve      |        
  Mat32.lengthscale        |  5.97926956859  |      +ve      |        
  Gaussian_noise.variance  |  17279.1887841  |      +ve      |        
('Optimise GPR for:', 'RBF', 5)
0


 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/rbf.py:35: RuntimeWarning:overflow encountered in square
 /home/riddhisw/anaconda2/lib/python2.7/site-packages/GPy/kern/src/rbf.py:38: RuntimeWarning:invalid value encountered in multiply


Optimisation complete!

Name : gp
Objective : 15217.2873931
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |          value  |  constraints  |  priors
  rbf.variance             |  45443.3464574  |      +ve      |        
  rbf.lengthscale          |  6.60668569066  |      +ve      |        
  Gaussian_noise.variance  |  36238.6071725  |      +ve      |        
('Optimise GPR for:', 'RQ', 5)
0
('Unexpected error:', <class 'numpy.linalg.linalg.LinAlgError'>)
('Failed run', 5)
('Optimise GPR for:', 'MAT32', 5)
0
Optimisation complete!

Name : gp
Objective : 3581.12392046
Number of Parameters : 3
Number of Optimization Parameters : 3
Updates : True
Parameters:
  gp.                      |               value  |  constraints  |  priors
  Mat32.variance           |       781419.616238  |      +ve      |        
  Mat32.lengthscale        |       2.56260258371  |      +ve      |        
  Gaussian_noise.variance  |  5.5626